In [ ]:
# Included in Python standard libraries
import time
import random

# Install these via a package manager
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from ipywidgets import interactive  # From Jupyter



%matplotlib inline

# Set matplotlib DPI for clearer pictures
mpl.rcParams['figure.dpi'] = 300

# Comment this out if you get errors when plotting, these are just my plotting
# preferences and are not required for the notebook
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.family'] = 'STIXGeneral'
mpl.rcParams['text.usetex'] = True

# Computing $\pi$ with `Python`

In this short tutorial, you'll learn about how to "numerically" compute $\pi = 3.14...$ using random sampling. We will do this using a very popular technique called [Monte Carlo](https://en.wikipedia.org/wiki/Monte_Carlo_method), which generally speaking *rely on random sampling to produce results*. After this tutorial, you will be able to 
1. understand the utility of random sampling in the numerical calculation
2. write efficient numerical `Python` code
3. visualize your results using `matplotlib`

## Background

### Relationship between the radius and circumference of a circle

Let's pretend for a moment that you've forgotten everything you know about $\pi$ except that it is the ratio of the circumference ($C$) of a circle to it's diameter ($d$):

$$ \pi = \frac{C}{d}.$$

Importantly, above I am **defining pi**. I am pretending to not know it's value, and I am suggesting that the ratio between the circumference and diameter of a circle is constant.

You also know that the diameter is simply two times the radius: $d = 2r.$ Rearranging the above equation, you can write the form for the circumference of a circle in terms of its radius, the number 2, and the constant $\pi$ (which remember, you've forgotten its value!),

$$ C = 2 \pi r. $$

This equation should be familiar to you!

### Areas of circles and squares

The area of a square of length $L$ is of course $L^2$:

$$ A_\mathrm{square}(L) = L^2.$$

What is the area of a circle?

---

**If you do not know calculus, skip this part!**

<font color='red'>
From calculus, we can actually derive the area of a circle from the circumference:

$$ A_\mathrm{circle}(r) = 2 \pi \int_0^r x \: dx = \pi r^2$$
</font>

---

Regardless, you should know anyway that the area of a circle of radius $r$ is $\pi r^2$:

$$A_\mathrm{circle}(r) = \pi r^2.$$

Note that so far, we did not need to know what the *value* of $\pi$ is yet!


### A circle inscribed in a square

Take a moment, and convince yourselves that *geometrically* the figure below makes sense. The circle (orange) is inscribed within the square (black). If the circle has radius $r,$ then the square must have side length $2r.$

![title](img/circle_in_square.png)

This means that the area of the area of the square and the circle are given by

$$ A_\mathrm{square}(r) = 4r^2, \quad A_\mathrm{circle}(r) = \pi r^2. $$

### The area ratio

Lets take a look at what happens if we compute the ratio of the area of a circle to a square:

$$ \frac{A_\mathrm{circle}}{A_\mathrm{square}} = \frac{\pi}{4}.$$

If we have some way of computing the ratio of the areas, we will be able to figure out what $\pi$ is. Fortunately, this is possible. We can use Monte Carlo sampling to do so.

### Random sampling

Consider the following experiment: you throw a dart randomly somewhere within the black square above. This dart will always land within the square, but it may or may not land within the orange circle. Note that the probability of landing in the circle is given by the area ratio above. Thus, if one takes a single random point, uniformally distributed in the square, the probability of landing within the circle is $\pi/4.$ Moreover, we can conclude if we take an infinite number of points, the ratio of the total number of points that fall within the circle, to that of the total number of points sampled will equal $\pi/4.$ This is the essense of Monte Carlo sampling. We can leverage the idea of randomness in smart ways to compute quantities of interest.

In [ ]:
random_x = np.random.uniform(low=-1.0, high=1.0, size=5000)
random_y = np.random.uniform(low=-1.0, high=1.0, size=5000)
color = random_x**2 + random_y**2 < 1.0
color = ['red' if c == 1 else 'black' for c in color]

In [ ]:
plt.clf()
def f(npts):
    fig, axs = plt.subplots(1, 1, figsize=(2, 2))
    axs.scatter(random_x[:npts], random_y[:npts], color=color[:npts], s=0.1)
    axs.set_xlim(-1, 1)
    axs.set_ylim(-1, 1)
    axs.set_aspect("equal")
    fig.tight_layout()
    plt.show()

interactive_plot = interactive(f, npts=(0, 5000, 100))
interactive_plot

## Core code and results

To simply make the usage of the code easier, lets define a class which contains everything we need to perform the sampling.

In [ ]:
class Sampler:
    
    def __init__(self, N):
        self.N = N
        
    def sample(self):
        
        # First, we sample uniformally on the x and y axes; note
        # that the length of the side of the square is 2, meaning
        # the radius of the inscribed circle is 1.
        x = np.random.uniform(low=-1.0, high=1.0, size=self.N)
        y = np.random.uniform(low=-1.0, high=1.0, size=self.N)
        return x, y
    
    def p_in_unit_circle(self):
        
        x, y = self.sample()
        
        # Next, we check if each (x, y) point is within the
        # circle. Recall that the equation for a circle is
        # x^2 + y^2 = 1 (for radius 1). A point is inside the
        # circle if x^2 + y^2 < 1, which we can easily check
        # using numpy vectorization
        in_circle = x**2 + y**2 < 1.0
        
        # True values evaluate to 1, and False evalutes to 0. So,
        # The sum of in_circle gives us the total number of points
        # that fell in the circle. Dividing by N gives us the ratio
        return in_circle.sum() / self.N
        
    def predict_pi(self, n_samples=1):
        
        # Return the computed value over n_samples Monte Carlo simulations,
        # in addition to the standard error in the mean
        samples = np.array([self.p_in_unit_circle() * 4.0 for _ in range(n_samples)])
        return samples.mean(), samples.std() / np.sqrt(self.N)

### Plotting the `x` and `y` sampling

Before analyzing what our code is predicting, we can first plot some examples of what the samples look like as a function of the total number of points sampled in the simulation. Before that though, let's just confirm our intuition about the random sampling of `x` and `y`.

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(4, 2), sharex=True, sharey=True)

sampler = Sampler(10000)
x, y = sampler.sample()

# Histogram
h = axs[0].hist2d(x, y, bins=50)
axs[0].set_xlim(-1, 1)
axs[0].set_ylim(-1, 1)
axs[0].set_title("2D Histogram", fontsize=6)
axs[0].set_aspect("equal")

# Scatter plot
# circle1 = plt.Circle((0, 0), 1, color='r', alpha=0.1)
# plt.gca().add_patch(circle1)
axs[1].scatter(x, y, s=0.001, color='black')
axs[1].set_xlim(-1, 1)
axs[1].set_ylim(-1, 1)
axs[1].set_title("Scatterplot", fontsize=6)
axs[1].set_aspect("equal")


plt.show()

Looks like random noise, as expected.

### Monte Carlo results

Now, we can look at how the results change as a function of the number of samples.

In [ ]:
N = 10**np.array([ii for ii in range(1, 7)])

In [ ]:
predictions = []
for nn in N:
    sampler = Sampler(nn)
    (pred, _) = sampler.predict_pi()
    predictions.append(pred)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))
ax.plot(N, predictions, 'ro')
ax.axhline(np.pi, *ax.get_xlim(), color='black', label=r"$\pi$")
ax.set_xscale('log')
ax.set_ylabel(r"$\pi$~(approximated)")
ax.set_xlabel(r"$N$")
plt.legend()
plt.show()

### Error analysis

It is always important to analyze how the error in the predicted result as a function of the number of samples.

In [ ]:
N = 10**np.array([ii for ii in range(1, 9)])
predictions = []
for nn in N:
    sampler = Sampler(nn)
    (pred, _) = sampler.predict_pi(1)
    predictions.append(np.abs(pred - np.pi))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))
ax.plot(np.log10(N), np.log10(predictions), 'ro')
ax.plot(np.log10(N), np.log10(1.0 / np.sqrt(N)), 'k')
ax.set_ylabel(r"$\log_{10} |\pi_\mathrm{true} - \pi_\mathrm{pred}|$")
ax.set_xlabel(r"$\log_{10} N$")
plt.show()

Log-log plot. What happens when we plot $y = 1/x$ on a log-log scale?

$$ \log y = \log (1/\sqrt{x}) = \log 1 - \frac{1}{2}\log x $$

If you define new variables, $x_\mathrm{log} = \log_{10} x$ and $y_\mathrm{log} = \log_{10} y$ and plot those (this is what we're doing) you can see those variables have a linear relationship, with a slope of $-1/2.$

## Analysis of sampling efficiency of different methods

If you are used to compiled computer languages, you might be tempted to do something like this when sampling random numbers:

In [ ]:
samples = []
for ii in range(1000):
    samples.append(random.random())  # Fill list with random samples between 0 and 1

However, you'll note that in all the code in this notebook, there is not a single for loop which iterates over more than 10 or so values. Why did I choose to do this?

**Python for loops are extremely slow.** In numerical scientific computing, we avoid Python for loops at all costs.

The `np.random.uniform` function calls fast low-level code backends, and does not compute the sample using Python for loops. Thus, it is very fast. We can quantify this:

In [ ]:
def slow_sample(N, avg_over=10):
    """Samples N uniform random numbers between 0 and 1.
    Returns the mean time it took to run and the standard deviation."""

    all_times = []
    for _ in range(avg_over):
        t0 = time.time()
        samples = []
        for ii in range(N):
            samples.append(random.random())  # Fill list with random samples between 0 and 1
        all_times.append(time.time() - t0)
    return np.mean(all_times), np.std(all_times)

def fast_sample(N, avg_over=10):
    """Does the same thing as slow_sample, but using numpy."""
    
    all_times = []
    for _ in range(avg_over):
        t0 = time.time()
        np.random.uniform(low=0.0, high=1.0, size=N)
        all_times.append(time.time() - t0)
    return np.mean(all_times), np.std(all_times)

In [ ]:
N = 10**np.array([ii for ii in range(7)])

slow_sample_runtime = [slow_sample(nn) for nn in N]
slow_sample_means = [xx[0] for xx in slow_sample_runtime]
slow_sample_std = [xx[1] for xx in slow_sample_runtime]

fast_sample_runtime = [fast_sample(nn) for nn in N]
fast_sample_means = [xx[0] for xx in fast_sample_runtime]
fast_sample_std = [xx[1] for xx in fast_sample_runtime]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2))
ax.errorbar(N, slow_sample_means, yerr=slow_sample_std, color='red', label="For loop")
ax.errorbar(N, fast_sample_means, yerr=fast_sample_std, color='blue', label="Numpy")
ax.set_xscale('log')
ax.set_ylabel(r"Runtime (s)")
ax.set_xlabel(r"Number of samples")
ax.legend()
plt.show()